In [1]:
import matplotlib.pyplot as plt

In [2]:
import matplotlib.dates as mdates

In [3]:
import importlib

In [4]:
import sh_ssw_methods as ssw

In [5]:
import numpy as np
import pandas as pd

In [6]:
from config import params

- identify events

In [7]:
event_dates_all, defn_all = [], []

In [8]:
# zeof1_lim
ze_lv50 = ssw.zeof1_lim(**params["zeof1_lim_lv50"])
ze_lv10 = ssw.zeof1_lim(**params["zeof1_lim_lv10"])
ze_lv1  = ssw.zeof1_lim(**params["zeof1_lim_lv1"])
event_dates_all += [ze_lv50, ze_lv10, ze_lv1]
defn_all += ["zeof1_lim_lv50","zeof1_lim_lv10","zeof1_lim_lv1"]

Saved 28 events to EOF1.Z50.gt1.0.csv
Saved 29 events to EOF1.Z10.gt1.0.csv
Saved 29 events to EOF1.Z1.gt1.0.csv


In [9]:
# ozone_butler
o3 = ssw.ozone_butler(**params["ozone_butler"])
event_dates_all.append(o3); defn_all.append("ozone_butler")

sh_ssw_totcol_o3_40DU.csv saved.


In [ ]:
# uanom karpechko
ua10 = ssw.u_anom_karpechko(**params["u_anom_karpechko_10"])
ua50 = ssw.u_anom_karpechko(**params["u_anom_karpechko_50"])
event_dates_all += [ua10, ua50]; defn_all += ["u_anom_karpechko_10","u_anom_karpechko_50"]

['1982-10-21' '1986-09-06' '1988-08-30' '1988-09-25' '1990-09-28'
 '2000-10-18' '2002-08-22' '2004-09-28' '2007-09-19' '2008-08-05'
 '2009-07-23' '2012-08-20' '2012-10-08' '2013-09-19' '2017-09-20'
 '2019-09-01']
16 3.8095238095238093 42.0


In [ ]:
# utend kaprechko
ut10 = ssw.u_tend_karpechko(**params["u_tend_karpechko_10"])
ut50 = ssw.u_tend_karpechko(**params["u_tend_karpechko_50"])
event_dates_all += [ut10, ut50]; defn_all += ["u_tend_karpechko_10","u_tend_karpechko_50"]

In [ ]:
# polarT Shen
pt = ssw.polarT_shen(**params["polarT_shen"])
event_dates_all.append(pt); defn_all.append("polarT_shen")

In [ ]:
# EPflux Reichler
epf, dat = ssw.epflux_reichler(**params["epflux_reichler"])
event_dates_all.append(epf); defn_all.append("epflux_reichler")

In [ ]:
# udecel Wu
ud10, drops = ssw.u_decel_wu(**params["u_decel_wu_10"])
event_dates_all.append(ud10); defn_all.append("u_decel_wu_10")

In [ ]:
# # stmi Lim
# stmi_years = ssw.stmi_lim(**params["stmi_lim"])
# event_dates_all.append(stmi_years); defn_all.append("stmi_lim")

- some statistics and some plots

In [ ]:
def avg_events_per_decade(event_dates, clim_range):
    """
    Average number of events per decade, given a climatological range.
    """
    start = pd.to_datetime(clim_range[0]).year
    end   = pd.to_datetime(clim_range[1]).year
    n_years = end - start + 1
    
    n_events = len(event_dates)
    return (n_events / n_years) * 10.0

In [ ]:
def to_datetime_index(arr):
    """Best-effort conversion to a 1D pandas.DatetimeIndex."""
    # If a method returned a tuple (e.g., (events, meta)), take the first piece
    if isinstance(arr, tuple):
        arr = arr[0]

    # xarray DataArray -> .values (but don't require xarray import)
    if hasattr(arr, "values"):
        arr = arr.values

    # Convert to 1D numpy array
    arr = np.asarray(arr).ravel()

    # If already datetime64, let pandas handle it; else attempt parse
    return pd.to_datetime(arr)

In [ ]:
def plot_events_sorted(event_dates_all, defn_all, clim_range):
    # --- compute averages ---
    stats = []
    for label, dates in zip(defn_all, event_dates_all):
        avg_dec = avg_events_per_decade(dates, clim_range)
        stats.append((label, dates, avg_dec))

    # --- sort descending by avg events per decade ---
    stats_sorted = sorted(stats, key=lambda x: x[2], reverse=False)

    # --- plot ---
    fig, ax = plt.subplots(figsize=(15, 5))
    yticks, ylabels = [], []

    for ypos, (label, dates, avg_dec) in enumerate(stats_sorted, start=1):
        ax.vlines(dates, ypos - 0.4, ypos + 0.4, color='green', linewidth=2, alpha=0.9)
        yticks.append(ypos)
        ylabels.append(f"{label} ({avg_dec:.1f}/decade)")

    ax.set_yticks(yticks)
    ax.set_yticklabels(ylabels)

    # x-axis: years
    ax.xaxis.set_major_locator(mdates.YearLocator(5))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax.set_xlabel("Year")
    ax.set_title("Detected Events – Sorted by # events per decade")
    ax.grid(True, axis="x", linestyle=":", alpha=0.6)

    plt.tight_layout()
    plt.show()


# Example usage
clim_range = ['1979-01-01','2020-12-31']
plot_events_sorted(event_dates_all, defn_all, clim_range)
